In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [6]:
all_files_path=glob('Dataset/*.edf')
print(len(all_files_path))

28


In [13]:
all_files_path[0]

'Dataset/h07.edf'

In [14]:
healthy_file_path=[i for i in all_files_path if 'h' in i.split('/')[1]]
patient_file_path=[i for i in all_files_path if 's' in i.split('/')[1]]
print(len(healthy_file_path),len(patient_file_path))

14 14


In [21]:
def read_data(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [22]:
sample_data=read_data(healthy_file_path[0])

Extracting EDF parameters from /home/blackheart/Documents/DATA SCIENCE/DS_CODE_EXERCISE/MNE/Dataset/h07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


Not setting metadata
227 matching events found


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.5s finished


No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 227 events and 1250 original time points ...
0 bad epochs dropped


In [ ]:
def read_data(file_path):
    # Read the EEG data from the specified file using MNE-Python
    data = mne.io.read_raw_edf(file_path, preload=True)
    
    # Set the EEG reference to be the average of all channels
    data.set_eeg_reference()
    
    # Apply a bandpass filter to the EEG data between 0.5 and 45 Hz
    data.filter(l_freq=0.5, h_freq=45)
    
    # Divide the EEG data into epochs of fixed length (5 seconds) with 1 second overlap
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap=1)
    
    # Extract the EEG data from the epochs object and return it as a numpy array
    array = epochs.get_data()
    
    return array


In [23]:
sample_data.shape # no of epochs, channel, length of signal

(227, 19, 1250)

In [24]:
%%capture
control_epochs_array=[read_data(i) for i in healthy_file_path]
patient_epochs_array=[read_data(i) for i in patient_file_path]

In [25]:
# Labling both data by 0 and 1
control_epochs_array[0].shape,patient_epochs_array[0].shape

((227, 19, 1250), (271, 19, 1250))

In [27]:
cotrol_epoch_labels=[len(i)*[0] for i in control_epochs_array]
patient_epoch_labels=[len(i)*[1] for i in patient_epochs_array]
len(cotrol_epoch_labels),len(patient_epoch_labels)

(14, 14)

In [28]:
data_list=control_epochs_array+patient_epochs_array
label_list=cotrol_epoch_labels+patient_epoch_labels

In [29]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

28

In [ ]:
data_array=np.vstack(data_list)
label_array=np.hstack(group_list)